In [ ]:
!pip install -U polars duckdb pyarrow

## DuckDB 

DuckDB is an "in-process" SQL OLAP database management system.

* No depependencies (written in C++)
* No setup. No server (embedded)
* Support for complex queries, including window functions, CTEs, and subqueries
* Open Source (MIT License)

### Installation

On Python you can install DuckDB with pip:

```bash
pip install duckdb
```

Supporting libraries:


In [ ]:
!pip install jupysql

In [ ]:
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = True
%config SqlMagic.displaycon = True


In [ ]:
%sql duckdb:///:memory: select 42

In [ ]:
%%sql 

select 42

In [ ]:
%%sql

df_42 << select 42

In [ ]:
df_42

# Load Data


In [ ]:
%%sql

DROP TABLE IF EXISTS vehicles;
CREATE TABLE vehicles AS FROM read_csv_auto('vehicles-pd.csv')

In [ ]:
%%sql
select * from vehicles

## Use Python instead of Cell Magic


In [ ]:
# Note that this is a new connection, so the table is not available
import duckdb
con = duckdb.connect(':memory:')
con.execute('select * from vehicles').fetchdf()

In [ ]:
# Note that this is a new connection, so the table is not available
import duckdb
con = duckdb.connect('vehicle.db')
sql = '''DROP TABLE IF EXISTS vehicles;
CREATE TABLE vehicles AS FROM read_csv_auto('vehicles-pd.csv')'''
con.execute(sql)

In [ ]:
%%sql duckdb:///vehicle.db
select * from vehicles

In [ ]:
%%sql

select * from vehicles

## Show Complex Query

In [ ]:
%%sql 
DROP TABLE IF EXISTS origin_map;
CREATE TABLE origin_map AS (
    SELECT * FROM (VALUES 
        ('Chevrolet', 'USA'),
        ('Ford', 'USA'),
        ('Dodge', 'USA'),
        ('GMC', 'USA'),
        ('Toyota', 'Japan'),
        ('BMW', 'Germany'),
        ('Mercedes-Benz', 'Germany'),
        ('Nissan', 'Japan'),
        ('Volkswagen', 'Germany'),
        ('Mitsubishi', 'Japan'),
        ('Porsche', 'Germany'),
        ('Mazda', 'Japan'),
        ('Audi', 'Germany'),
        ('Honda', 'Japan'),
        ('Jeep', 'USA'),
        ('Pontiac', 'USA'),
        ('Subaru', 'Japan'),
        ('Volvo', 'Sweden'),
        ('Hyundai', 'South Korea'),
        ('Chrysler', 'USA'),
        ('Tesla', 'USA')
    ) AS t(make, origin)
);



In [ ]:
%%sql

WITH processed_data AS (
    SELECT 
        v.make,
        v.model,
        v.year,
        v.city08,
        v.highway08,
        COALESCE(om.origin, 'Unknown') AS origin,
        STRPTIME(v.createdOn, '%a %b %d %H:%M:%S %Z %Y') AS createdOn
    FROM vehicles AS v
    LEFT JOIN origin_map AS om ON v.make = om.make
    WHERE COALESCE(om.origin, 'Unknown') != 'Unknown' AND v.year < 2020
),
 agg_data AS(
    SELECT 
        year, 
        origin,
        AVG(city08) AS avg_city08
    FROM processed_data
    GROUP BY year, origin
    ORDER BY year, origin
)

PIVOT agg_data
ON origin
USING mean(avg_city08)
GROUP BY year


In [ ]:
res = _
res

In [ ]:
(res
 .set_index('year')
 .plot()
)

## Exponential Growth

In [ ]:
# I invest $1 and it grows by 1% every day for 1,000 days
# how would I calculate the value with pandas?
import pandas as pd
investment = pd.Series([1])

def compound_growth(start, rate, periods):
    result = pd.Series([start]*periods, dtype='float64[pyarrow]')
    for i in range(1, periods+1):
        result[i] = result[i-1] * (1+rate)
    return result

compound_growth(1, 0.01, 10_000)

# Here is the equivalent SQL

query = '''WITH RECURSIVE CompoundGrowth(day, value) AS (
    SELECT 1, 1.00 -- Initial investment of $1 on day 1
    UNION ALL
    SELECT day + 1, value * 1.01 -- Compound growth of 1% per day
    FROM CompoundGrowth
    WHERE day < 10000
)
SELECT * FROM CompoundGrowth;
'''

In [ ]:
import duckdb
con_mem = duckdb.connect(':memory:')
query = '''WITH RECURSIVE CompoundGrowth(day, value) AS (
    SELECT 1, 1.00::DOUBLE -- Initial investment of $1 on day 1
    UNION ALL
    SELECT day + 1, value * 1.01 -- Compound growth of 1% per day
    FROM CompoundGrowth
    WHERE day < 10000
)
SELECT * FROM CompoundGrowth;
'''
con_mem.sql(query).df()

## Arrow Integration

In [ ]:
# read from polars
import polars as pl
df_pl = pl.read_csv('vehicles-pd.csv')
duckdb.sql('select * from df_pl').show()


In [ ]:
# Convert output to polars
duckdb.sql('SELECT * FROM df_pl LIMIT 2').pl()

In [ ]:
# Read from pandas
import pandas as pd
import duckdb
df_pd = pd.read_csv('vehicles-pd.csv', dtype_backend='pyarrow')
duckdb.sql('select * from df_pd').show()


In [ ]:
# Convert results to Pandas (not using pyarrow)
df_pd = pd.read_csv('vehicles-pd.csv', dtype_backend='pyarrow')
duckdb.sql('select * from df_pd').df()


## DuckDB Exercises

1. **Basic DataFrame Operations**
   - Show the shape of the vehicles Database. (Hint: `COUNT(*)` is your friend)
   - Print the first 5 rows. (Hint: `LIMIT` is your friend)
   - Print the last 5 rows. (Hint: not really possible in DuckDB as rows are not ordered. Ok, it kind of is with `OFFSET` and `LIMIT`)
   - Print the list of columns. (Hint: See `information_schema.columns` table)

2. **Data Exploration**
   - Print the number of unique values in each column of the vehicles Database. (Hint: see `COUNT(DISTINCT column_name)` and don't worry about every column)
   - Print the number of null values in each column. (Hint: see `WHERE column IS NULL` and don't worry about every column, maybe try the cylinders column)
   - Print the mean and standard deviation of the 'city08' column. (Hint: see `AVG` and `STDDEV` functions)
   - Print the median and 75th percentile of the 'city08' column. (Hint: `PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY city08) AS median_city08`  or for an easier life, see `QUANTILE` function)

3. **String Manipulation**
   - Upper case the 'make' column (Hint: see `UPPER` function)
   - Combine the 'year' and 'make' columns into a new column called 'year_make'. (Hint: use `||` to concatenate strings)

4. **Datetime Conversion**
   - Convert the 'createdOn' column to the New York timezone. 
      - You can see the current timezone with  `SELECT * FROM duckdb_settings() WHERE name = 'TimeZone'`
      - You can set the timezone with  `SET TimeZone = 'America/New_York';`

5. **Data Filtering**
   - Filter data to only include rows where the 'make' column is 'Ford'.
   - Filter the data to only include rows where the 'model' column is a single word. (Hint: see `SPLIT_PART` function)
   - Filter the rows where the city mileage is greater than 75% of the city mileage values.

6. **Grouping and Aggregation (Moderate)**
   - Find the average mileage for Ford, Tesla, and Toyota vehicles. (Hint: see `GROUP BY` and `WHERE ... IN ... `)
   - Find the average mileage by year and make. Pivot for bonus points. (Bonus Hint: Create CTE with `WITH` and then `SELECT` from it with `PIVOT`)


# Summary

- Pandas 2 provides speed improvements over Pandas 1, but is still single threaded
- Polars is fast but a new syntax to learn
- DuckDB is fast and SQL is familiar, but not all SQL functions are available. You tend to write a lot of SQL to get the job done.

Contact me at [matt@metasnake.com] I'm on Twitter at @\_\_mharrison\_\_ 

I would love to teach your teams to use these tools effectively. Expect a Pandas 2 book from me soon.